## ticdat and pickle

`ticdat` objects aren't naturally `pickle` friendly, but they are also play nice with `pickle` (via json).

In [1]:
import ticdat
ticdat.__version__

'0.2.19'

### pulling some data and code from the ticdat gurobipy/diet example

In [2]:
import diet
dat = diet.input_schema.csv.create_tic_dat("diet_sample_data")
dat

td: {categories: 4, nutrition_quantities: 36, foods: 9}

In [3]:
import pickle
import json

### to get json.write_file to return a json string, just pass an empty file path
(oops, there is actually, a `to_str` argument there that is ignored, I'm a gonna remove/fix that in the next 
release)

In [4]:
the_dict_i_can_pickle = json.loads(diet.input_schema.json.write_file(dat, ""))
{k: len(v) for k, v in the_dict_i_can_pickle.items()}

{'categories': 4, 'foods': 9, 'nutrition_quantities': 36}

In [5]:
with open("dat.p", "wb") as f:
    pickle.dump(the_dict_i_can_pickle, f)

### now de-serialize it from the pickle
I'm going to do it all in one go so hang onto your butt

In [6]:
with open("dat.p", "rb") as f:
    dat2 = diet.input_schema.json.create_tic_dat(json.dumps(pickle.load(f)))
dat2

td: {categories: 4, nutrition_quantities: 36, foods: 9}

In [7]:
diet.input_schema._same_data(dat, dat2, epsilon=0.0001)

True

### now lets do it with PanDatFactory

First we munge everything over to clean `PanDat` objects. I'm going to do a couple more steps than are strictly needed in order to make a clearly "clean" `PanDat` object.

In [8]:
pdf = ticdat.PanDatFactory.create_from_full_schema(diet.input_schema.schema(include_ancillary_info=True))

In [9]:
pan_dat = pdf.copy_pan_dat(diet.input_schema.copy_to_pandas(dat, drop_pk_columns=False))
pan_dat

pd: {categories: 4, nutrition_quantities: 36, foods: 9}

In [10]:
the_dict_i_can_pickle = json.loads(pdf.json.write_file(pan_dat, ""))
{k: len(v) for k, v in the_dict_i_can_pickle.items()}

{'categories': 2, 'nutrition_quantities': 2, 'foods': 2}

In [11]:
with open("pan_dat.p", "wb") as f:
    pickle.dump(the_dict_i_can_pickle, f)

In [13]:
with open("pan_dat.p", "rb") as f:
    pan_dat2 = pdf.json.create_pan_dat(json.dumps(pickle.load(f)))
pan_dat2

pd: {categories: 4, nutrition_quantities: 36, foods: 9}

In [14]:
pdf._same_data(pan_dat, pan_dat2, epsilon=0.0001)

True